In [2]:
import os
import time
import pandas as pd
import numpy as np

import rasterio

import dask_gateway
import dask.array as da

from joblib import load

import raster_to_features as rf

import geopandas as gpd
from shapely.geometry import mapping

from scipy.ndimage import median_filter

In [3]:
# year for which to predict iceplant locations
year = 2020

# whether to apply median filter to raster
filter_rasters = True

# whether to save rasters
save_rasters = True

In [4]:
# open shapefile of SB coastal buffer and process it to use it for clipping
fp = os.path.join(os.getcwd(), 'separating_naip_flights', 'SB_coastal_buffer', 'SB_coastal_buffer.shp')
coast = gpd.read_file(fp)
coast_geo = coast.geometry.apply(mapping)

In [5]:
# open pre-trained random forest classifier
rfc = load('spectral_rfc.joblib') 
print('loaded model')

loaded model


In [6]:
# initialize DASK cluster
cluster = dask_gateway.GatewayCluster()
cluster.scale(30)

client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.c09e3a74a5e34cf880c3b96c637f6895/status,


In [7]:
# select the scene ids from given year that intersect the coastal buffer
# the itemids of all scenes that intersect the coast were previously stored in a csv
scene_ids = pd.read_csv(os.path.join(os.getcwd(),
                                     'separating_naip_flights',
                                     'coastal_scenes_ids.csv'))
scene_ids = scene_ids.loc[scene_ids['year'] == year]
scene_ids = scene_ids.reset_index().itemid
scene_ids

0     ca_m_3412037_nw_10_060_20200607
1     ca_m_3412037_ne_10_060_20200607
2     ca_m_3412029_sw_10_060_20200607
3     ca_m_3412004_sw_10_060_20200607
4     ca_m_3412003_se_10_060_20200607
5     ca_m_3412003_ne_10_060_20200607
6     ca_m_3412038_nw_10_060_20200523
7     ca_m_3412040_nw_10_060_20200522
8     ca_m_3412040_ne_10_060_20200522
9     ca_m_3412039_nw_10_060_20200522
10    ca_m_3412039_ne_10_060_20200522
11    ca_m_3412038_ne_10_060_20200522
12    ca_m_3411933_nw_11_060_20200522
13    ca_m_3411933_ne_11_060_20200522
14    ca_m_3411945_nw_11_060_20200521
15    ca_m_3411937_sw_11_060_20200521
16    ca_m_3411936_sw_11_060_20200521
17    ca_m_3411936_se_11_060_20200521
18    ca_m_3411935_sw_11_060_20200521
19    ca_m_3411935_se_11_060_20200521
20    ca_m_3411934_sw_11_060_20200521
21    ca_m_3411934_se_11_060_20200521
22    ca_m_3411933_sw_11_060_20200521
23    ca_m_3411933_se_11_060_20200521
Name: itemid, dtype: object

In [8]:
# ---------------------------------------
# collect processing information for each scene
times_pre = []
times_class = []
times_post = []
processed = []
reason = []
n_pixels = []

# counter for scenes queued for processing
N = len(scene_ids)

# ---------------------------------------
# prepare folder to save rasters
if save_rasters:
    fp = os.path.join(os.getcwd(),'temp')
    if os.path.exists(fp) == False:
        os.mkdir(fp)
    fp = os.path.join(fp, 'S_clip_preds_' + str(year))
    if os.path.exists(fp) == False:
        os.mkdir(fp)

        
# ---------------------------------------
# ---------------------------------------
for itemid in scene_ids:
    t_alpha = time.time()

    # ***********************************************************************************************
    # *************************************** PRE-PROCESSING ****************************************
    # ---------------------------------------
    # open NAIP scene and clip to coast
    t0 = time.time()
    raster = rf.rioxr_from_itemid(itemid).rio.clip(coast_geo,coast.crs)

    # ---------------------------------------
    # select pixels with data (blacked out portions have 0 on all bands)
    df = rf.raster_as_df(raster.to_numpy(), ['r','g','b','nir'])
    df = df.loc[ (df['nir'] != 0) | (df['r'] != 0) | (df['g'] != 0) | (df['b'] != 0)]

    # ---------------------------------------
    # stop if there's no data at intersection
    if df.shape[0] == 0:
        processed.append('N')
        reason.append('no data in intersection')
        times_pre.append(0)
        times_class.append(0)        
        times_post.append(0)
        n_pixels.append(0)
        N = N-1
        print('no data at intersection of scene with coastal buffer')
        print('FINISHED: ', itemid , '\n')

    else:
        # find vegetation pixels to go into model
        # keep indices of water and low-ndvi pixels
        # add ndvi and ndwi features for each pixel
        t0 = time.time()
        is_veg, water_index, not_veg_index = rf.add_spectral_features(df, 
                                                                      ndwi_thresh = 0.3, 
                                                                      ndvi_thresh = 0.05) 
        # ---------------------------------------
        # stop if there are no vegetation pixels at intersection
        if is_veg.shape[0] == 0:
            processed.append('N')
            reason.append('no vegeatation in intersection')
            times_pre.append(0)
            times_class.append(0)        
            times_post.append(0)
            n_pixels.append(0)
            N = N-1
            print('no vegetation pixels at intersection of scene data with coastal buffer')
            print('FINISHED: ', itemid , '\n')
            
        else:
            processed.append('Y')
            reason.append('processed')           
            
            # ---------------------------------------
            # select features
            is_veg.drop('ndwi', axis=1, inplace=True)
            is_veg = rf.add_date_features(is_veg, rf.rioxr_from_itemid(itemid).datetime)
            
            times_pre.append(time.time() - t0)
            n_pixels.append(len(is_veg))       
            print('finished pre-processing')
            
    # ***********************************************************************************************
    # *************************************** CLASSIFICATION ****************************************
            # ---------------------------------------
            # convert into dask.array and predict using model
            da_pixels = da.from_array(np.array(is_veg), chunks=728802)
            
            # ---------------------------------------
            scene_preds = rfc.predict(da_pixels)    

            t0 = time.time()
            preds = scene_preds.compute()
            times_class.append(time.time() - t0)
            print('finished classification')
    
    # ************************************************************************************************
    # *************************************** POST-PROCESSING ****************************************
            # ---------------------------------------
            # recover pixel indices for iceplant classifications
            t0 = time.time()
            preds_df = pd.DataFrame(preds, 
                                 columns=['is_iceplant'], 
                                 index = is_veg.index)
            is_iceplant_index = preds_df[preds_df.is_iceplant == 1].index.to_numpy()
            non_iceplant_index = preds_df[preds_df.is_iceplant == 0].index.to_numpy()

            # ---------------------------------------
            # reconstruct indices into image
            indices = [non_iceplant_index,
                       is_iceplant_index, 
                       not_veg_index,
                       water_index]
            values = [0,    # values assigned to pixels from each index
                      1,
                      2,
                      3]
            results = rf.indices_to_image(raster.shape[1], raster.shape[2], indices, values, back_value=100)
            
            # ---------------------------------------
            # apply median 3x3 filter if needed
            if filter_rasters:
                results = median_filter(results, size=3)
            
            times_post.append(time.time() - t0)
            print('finished post-processing')

    # ************************************************************************************************
    # *************************************** SAVING RASTERS ****************************************            

            # ---------------------------------------
            # save raster 
            if save_rasters:
                if filter_rasters:
                    filename = 'S_filter_clip_preds_' + itemid +'.tif'
                else:
                    filename = 'S_clip_preds_' + itemid +'.tif'
                    
                with rasterio.open(
                    os.path.join(fp, filename),  # file path
                    'w',           # w = write
                    driver = 'GTiff', # format
                    height = results.shape[0], 
                    width = results.shape[1],
                    count = 1,  # number of raster bands in the dataset
                    dtype = rasterio.uint8,
                    crs = raster.rio.crs,
                    transform = raster.rio.transform(),
                ) as dst:
                    dst.write(results.astype(rasterio.uint8), 1)

            # ---------------------------------------
            # info message
            N = N-1
            print('FINISHED: ', itemid)
            print('REMAINING: ', N, 'scenes \n')

finished pre-processing
finished classification
finished post-processing
FINISHED:  ca_m_3412037_nw_10_060_20200607
REMAINING:  23 scenes 

finished pre-processing
finished classification
finished post-processing
FINISHED:  ca_m_3412037_ne_10_060_20200607
REMAINING:  22 scenes 

finished pre-processing
finished classification
finished post-processing
FINISHED:  ca_m_3412029_sw_10_060_20200607
REMAINING:  21 scenes 

no data at intersection of scene with coastal buffer
FINISHED:  ca_m_3412004_sw_10_060_20200607 

no vegetation pixels at intersection of scene data with coastal buffer
FINISHED:  ca_m_3412003_se_10_060_20200607 

finished pre-processing
finished classification
finished post-processing
FINISHED:  ca_m_3412003_ne_10_060_20200607
REMAINING:  20 scenes 

finished pre-processing
finished classification
finished post-processing
FINISHED:  ca_m_3412038_nw_10_060_20200523
REMAINING:  19 scenes 

finished pre-processing
finished classification
finished post-processing
FINISHED:  ca

In [9]:
#save times processed and itemids as dataframe
D = { 'itemid': scene_ids,
     'processed': processed,
     'reason':reason,
     'pre_times': times_pre,
     'class_times' : times_class,
     'post_times' : times_post, 
     'n_pixels' : n_pixels}
processing_df = pd.DataFrame( D )

fp = os.path.join(os.getcwd(),'processing_results')
if os.path.exists(fp) == False:
    os.mkdir(fp)

processing_df.to_csv(os.path.join(fp, 'S_clip_processing_results_' + str(year) + '.csv'), index=False)